In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Install InstaGeo

In [4]:
## EXECUTE
# Clone the InstaGeo-E2E-Geospatial-ML repository from GitHub
repository_url = "https://github.com/instadeepai/InstaGeo-E2E-Geospatial-ML"
!git clone {repository_url}

fatal: destination path 'InstaGeo-E2E-Geospatial-ML' already exists and is not an empty directory.


In [5]:

%%bash
cd InstaGeo-E2E-Geospatial-ML
git pull

From https://github.com/instadeepai/InstaGeo-E2E-Geospatial-ML
   fb475e7..305cf40  21-demo-new-design-tileserver-integration -> origin/21-demo-new-design-tileserver-integration


Already up to date.


/Users/mayaraayat/Desktop/geoAI/myenv/lib/python3.11/site-packages/IPython/core/completerlib.py:371: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bks = self.db.get('bookmarks',{})


In [7]:

%%capture
%%bash
# Navigate to the cloned InstaGeo-E2E-Geospatial-ML directory
cd InstaGeo-E2E-Geospatial-ML
# Stash any local changes to avoid conflicts when switching branches
git stash
#Switch to the 'geo-ai-hack' branch, which likely contains specific code for the Geo AI Hackathon
git checkout geo-ai-hack
# Install the InstaGeo package 
pip install -e .[all]

In [8]:
cd InstaGeo-E2E-Geospatial-ML

/Users/mayaraayat/Desktop/geoAI/InstaGeo-E2E-Geospatial-ML


/Users/mayaraayat/Desktop/geoAI/myenv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
## EXECUTE
# Import necessary libraries
import os
import re
import shutil
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
from pyproj import CRS, Transformer
import rasterio
os.environ["HYDRA_FULL_ERROR"] ="1"


In [9]:
## EXECUTE
def generate_label_mapping(root_dir, input_subdir, output_csv, start_year=20160101):
    """
    Generate a CSV mapping input chips to corresponding segmentation maps.

    Args:
        root_dir (str or Path): Root directory containing the subdirectories for chips and segmentation maps.
        input_subdir (str): Subdirectory path for chips within the root directory.
        output_csv (str or Path): Output path for the generated CSV file.
    """
    root_dir = Path(root_dir)
    chips_orig = os.listdir(root_dir / input_subdir / "chips")
    
    if os.path.exists(root_dir / input_subdir / "seg_maps"):
        add_label = True
    else:
        add_label = False

    
    chips_orig = [
        chip for chip in chips_orig
        if (chip.startswith("chip_") and int(chip.split("_")[1][:8]) >= start_year)
    ]
    chips = [chip.replace("chip", f"{input_subdir}/chips/chip") for chip in chips_orig]

    if add_label:
        seg_maps = [chip.replace("chip", f"{input_subdir}/seg_maps/seg_map") for chip in chips_orig]
        df = pd.DataFrame({"Input": chips, "Label": seg_maps})
    else:
        df = pd.DataFrame({"Input": chips})
    df.to_csv(output_csv, index=False)
    
    print(f"Number of rows is: {df.shape[0]}")
    print(f"CSV generated and saved to: {output_csv}")




In [10]:
## EXECUTE
# set data folder path
input_dir="/kaggle/input/geo-ai-hack"

In [11]:
## EXECUTE
# Generate label mappings for the training and testing datasets
generate_label_mapping(input_dir, 'hls_train/hls_train', "train_ds.csv",20180101)
generate_label_mapping(input_dir, 'hls_test/hls_test', "test_ds.csv")

Number of rows is: 8411
CSV generated and saved to: train_ds.csv
Number of rows is: 2404
CSV generated and saved to: test_ds.csv


### Validation Set

In [12]:
## EXECUTE
def split_validation_data(mapping_csv, validation_split=0.3):
    """
    Split data into training and validation sets based on a CSV file mapping `chips` and `seg_maps`.

    Args:
        mapping_csv (str or Path): Path to the CSV file containing the mapping between `chips` and `seg_maps`.
        data_dir (str or Path): Path to the merged directory containing all files.
        validation_dir (str or Path): Path to the new directory for validation files.
        validation_split (float): Fraction of the data to use as the validation set.
    """
    df = pd.read_csv(mapping_csv)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    num_val = int(len(df) * validation_split)
    train_df = df[num_val:]
    val_df = df[:num_val]
    train_df.to_csv("train_split.csv",index=False)    
    print(f"CSV train split  saved to: train_split.csv")
    val_df.to_csv("validation_split.csv",index=False)    
    print(f"CSV validation split  saved to: validation_split.csv")
    
    return 
    

In [13]:
## EXECUTE
# Split the training dataset into training and validation sets
split_validation_data(
    mapping_csv="train_ds.csv",
    validation_split=0.3
)

CSV train split  saved to: train_split.csv
CSV validation split  saved to: validation_split.csv


## InstaGeo - Model

After creating our training and validation splits, we can move on to fine-tuning a model that includes a Prithvi backbone paired with a classification head. For regression tasks, the classification head can easily be replaced with a suitable regression head. Additionally, if a completely different model architecture is needed, it can be designed and implemented within this framework.

In [ ]:
def load_yml(filepath):
    """Load data from a YAML file.

    Args:
        filepath (str | Path): The path to the YAML file.

    Returns:
        Dict: The loaded data, or None if the file does not exist.
    """
    filepath=Path(filepath)
    with filepath.open() as f:
        return yaml.safe_load(f)
        
def save_yml(data,filepath):
    """Save data to a YAML file.

    Args:
        data (Dict): The data to save.
        filepath (str | Path): The file path to save the YAML to.
    """
    filepath = Path(filepath)
    with filepath.open("w") as f:
        yaml.dump(data, f)
    print(f"Data saved to {filepath}.")


### Launch Training


First, compute the mean and standard deviation for the dataset using the InstaGeo command. Then update the corresponding configuration file [locust.yaml](https://github.com/instadeepai/InstaGeo-E2E-Geospatial-ML/blob/main/instageo/model/configs/locust.yaml). In this case, it has already been done for you. However, if you change the dataset split or modify the training data, you should run the command again to compute the new mean and standard deviation.

In [ ]:
# %%bash
# python -m instageo.model.run --config-name=locust \
#     root_dir="/kaggle/input/geo-ai-hack" \
#     train.batch_size=8 \
#     train.num_epochs=5 \
#     mode=stats \
#     train_filepath="train_ds.csv" \

In [ ]:
# Updat locust file
# Load the Locust model configuration file
locust_cfg_path="InstaGeo-E2E-Geospatial-ML/instageo/model/configs/locust.yaml"
# Load the YAML configuration into a dictionary
locust_cfg=load_yml(locust_cfg_path)
# Update the mean and standard deviation values in the configuration
locust_cfg["dataloader"]["mean"]=[670.5441284179688, 1267.7974853515625, 1772.599365234375, 2415.69091796875, 2879.2431640625, 2337.822509765625]
locust_cfg["dataloader"]["std"]=[2146.305419921875, 2203.416259765625, 2247.03515625, 2310.74755859375, 2322.708984375, 2211.968505859375]
# Save the updated configuration back to the YAML file
save_yml(locust_cfg,locust_cfg_path)

In [ ]:
# Train the InstaGeo model using the Locust configuration
!python -m instageo.model.run  --config-name=locust \
    hydra.run.dir="/kaggle/working/outputs/first_run" \
    root_dir="/kaggle/input/geo-ai-hack" \
    train.batch_size=12 \
    train.num_epochs=100 \
    mode=train \
    train_filepath="train_split.csv" \
    valid_filepath="validation_split.csv"

### Run Model Evaluation
 To evaluate the model, adjust the `checkpoint_path` argument to point to the desired model checkpoint. The checkpoint file is typically located in the `hydra.run.dir` directory and is named `instageo_best_checkpoint.ckpt`.
For example:
```
/kaggle/working/outputs/first_run/instageo_best_checkpoint.ckpt 
```
Make sure to provide the correct path to the checkpoint file based on your training output directory.

In [ ]:
%%bash
python -m instageo.model.run --config-name=locust \
    root_dir="/kaggle/input/geo-ai-hack" \
    test_filepath="validation_split.csv" \
    train.batch_size=16 \
    checkpoint_path='/kaggle/working/outputs/first_run/instageo_best_checkpoint.ckpt' \
    mode=eval

## Make Submission

We first run inference on test chips to get the predictions

In [10]:
%%bash
python -m instageo.model.run --config-name=locust \
    root_dir="/kaggle/input/geo-ai-hack" \
    test_filepath="test_ds.csv" \
    train.batch_size=16 \
    checkpoint_path='/kaggle/input/check/pytorch/default/1/instageo_best_checkpoint.ckpt' \
    output_dir='/kaggle/working/predictions' \
    mode=chip_inference

[2025-02-05 11:19:22,757][__main__][INFO] - Script: /kaggle/working/InstaGeo-E2E-Geospatial-ML/instageo/model/run.py
[2025-02-05 11:19:22,760][__main__][INFO] - Imported hydra config:
root_dir: /kaggle/input/geo-ai-hack
valid_filepath: null
train_filepath: null
test_filepath: test_ds.csv
checkpoint_path: /kaggle/input/check/pytorch/default/1/instageo_best_checkpoint.ckpt
output_dir: /kaggle/working/predictions
mode: chip_inference
train:
  learning_rate: 0.0001
  num_epochs: 20
  batch_size: 16
  class_weights:
  - 1
  - 1
  ignore_index: -1
  weight_decay: 0.1
model:
  freeze_backbone: false
  num_classes: 2
dataloader:
  bands:
  - 0
  - 1
  - 2
  - 3
  - 4
  - 5
  - 6
  - 7
  - 8
  - 9
  - 10
  - 11
  - 12
  - 13
  - 14
  - 15
  - 16
  - 17
  mean:
  - 623.2724609375
  - 1247.657958984375
  - 1772.24169921875
  - 2371.256103515625
  - 2862.867431640625
  - 2357.759765625
  std:
  - 2182.050048828125
  - 2248.420654296875
  - 2302.53515625
  - 2372.204345703125
  - 2398.52685546875
 

/kaggle/working/InstaGeo-E2E-Geospatial-ML/instageo/model/model.py:155: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_path, map_location="cpu

After getting the prdictions for each chip, we retrieve the predicted value for each observatio in our test split.

In [11]:


predictions_directory = "/kaggle/working/predictions"
prediction_files = os.listdir(predictions_directory)

def get_prediction_value(row):
    matching_files = [f for f in prediction_files if (str(row['date']) in f) and (row['mgrs_tile_id'] in f)]
    if not matching_files:
        return (np.nan, np.nan)
    for file in matching_files:
        with rasterio.open(f"{predictions_directory}/{file}") as src:
            width, height = src.width, src.height
            affine_transform = rasterio.transform.AffineTransformer(src.transform)
            transformer = Transformer.from_crs(CRS.from_epsg(4326), src.crs, always_xy=True)
            x_chip, y_chip = transformer.transform(row['x'], row['y'])
            x_offset, y_offset = affine_transform.rowcol(x_chip, y_chip)
            
            if 0 <= x_offset < width and 0 <= y_offset < height:
                return src.read(1)[y_offset, x_offset], file
    return (np.nan, np.nan)

In [12]:
submission_df = pd.read_csv("/kaggle/input/geo-ai-hack/test.csv")

submission_df[['prediction', 'filename']] = submission_df.apply(get_prediction_value, axis=1, result_type='expand')
submission_df[["id","prediction"]].to_csv("hls_submission.csv",index=False)

In [10]:
## EXECUTE

# Code inspired and modified from the InstaGeo-E2E-Geospatial-ML repository

"""Dataloader Module."""

import os
import random
from functools import partial
from typing import Callable, List, Tuple

import numpy as np
import sys
import pandas as pd
import rasterio
import torch
from absl import logging
from PIL import Image
from torchvision import transforms
import rioxarray as rxr
import xarray as xr

def open_mf_tiff_dataset(
    band_files: dict, load_masks: bool
) -> tuple[xr.Dataset, xr.Dataset | None, CRS]:
    """Open multiple TIFF files as an xarray Dataset.

    Args:
        band_files (Dict[str, Dict[str, str]]): A dictionary mapping band names to file paths.
        load_masks (bool): Whether or not to load the masks files.

    Returns:
        (xr.Dataset, xr.Dataset | None, CRS): A tuple of xarray Dataset combining data from all the
            provided TIFF files, (optionally) the masks, and the CRS
    """
    band_paths = list(band_files["tiles"].values())
    bands_dataset = xr.open_mfdataset(
        band_paths,
        concat_dim="band",
        combine="nested",
        mask_and_scale=False,  # Scaling will be applied manually
    )
    bands_dataset.band_data.attrs["scale_factor"] = 1
    mask_paths = list(band_files["fmasks"].values())
    mask_dataset = (
        xr.open_mfdataset(
            mask_paths,
            concat_dim="band",
            combine="nested",
        )
        if load_masks
        else None
    )
    with rasterio.open(band_paths[0]) as src:
        crs = src.crs
    return bands_dataset, mask_dataset, crs


def random_crop_and_flip(
    ims: List[Image.Image], label: Image.Image, im_size: int
) -> Tuple[List[Image.Image], Image.Image]:
    """Apply random cropping and flipping transformations to the given images and label.

    Args:
        ims (List[Image.Image]): List of PIL Image objects representing the images.
        label (Image.Image): A PIL Image object representing the label.

    Returns:
        Tuple[List[Image.Image], Image.Image]: A tuple containing the transformed list of
        images and label.
    """
    i, j, h, w = transforms.RandomCrop.get_params(ims[0], (im_size, im_size))

    ims = [transforms.functional.crop(im, i, j, h, w) for im in ims]
    label = transforms.functional.crop(label, i, j, h, w)

    if random.random() > 0.5:
        ims = [transforms.functional.hflip(im) for im in ims]
        label = transforms.functional.hflip(label)

    if random.random() > 0.5:
        ims = [transforms.functional.vflip(im) for im in ims]
        label = transforms.functional.vflip(label)

    return ims, label


def normalize_and_convert_to_tensor(
    ims: List[Image.Image],
    label: Image.Image | None,
    mean: List[float],
    std: List[float],
    temporal_size: int = 1,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """Normalize the images and label and convert them to PyTorch tensors.

    Args:
        ims (List[Image.Image]): List of PIL Image objects representing the images.
        label (Image.Image | None): A PIL Image object representing the label.
        mean (List[float]): The mean of each channel in the image
        std (List[float]): The standard deviation of each channel in the image
        temporal_size: The number of temporal steps

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: A tuple of tensors representing the normalized
        images and label.
    """
    norm = transforms.Normalize(mean, std)
    ims_tensor = torch.stack([transforms.ToTensor()(im).squeeze() for im in ims])
    _, h, w = ims_tensor.shape
    ims_tensor = ims_tensor.reshape([temporal_size, -1, h, w])  # T*C,H,W -> T,C,H,W
    ims_tensor = torch.stack([norm(im) for im in ims_tensor]).permute(
        [1, 0, 2, 3]
    )  # T,C,H,W -> C,T,H,W
    if label:
        label = torch.from_numpy(np.array(label)).squeeze()
    return ims_tensor, label


def process_and_augment(
    x: np.ndarray,
    y: np.ndarray | None,
    mean: List[float],
    std: List[float],
    temporal_size: int = 1,
    im_size: int = 224,
    augment: bool = True,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """Process and augment the given images and labels.

    Args:
        x (np.ndarray): Numpy array representing the images.
        y (np.ndarray): Numpy array representing the label.
        mean (List[float]): The mean of each channel in the image
        std (List[float]): The standard deviation of each channel in the image
        temporal_size: The number of temporal steps
        augment: Flag to perform augmentations in training mode.

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: A tuple of tensors representing the processed
        and augmented images and label.
    """
    ims = x.copy()
    label = None
    # convert to PIL for easier transforms
    ims = [Image.fromarray(im) for im in ims]
    if y is not None:
        label = Image.fromarray(y.copy().squeeze())
    if augment:
        ims, label = random_crop_and_flip(ims, label, im_size)
    ims, label = normalize_and_convert_to_tensor(ims, label, mean, std, temporal_size)
    return ims, label


def crop_array(
    arr: np.ndarray, left: int, top: int, right: int, bottom: int
) -> np.ndarray:
    """Crop Numpy Image.

    Crop a given array (image) using specified left, top, right, and bottom indices.

    This function supports cropping both grayscale (2D) and color (3D) images.

    Args:
        arr (np.ndarray): The input array (image) to be cropped.
        left (int): The left boundary index for cropping.
        top (int): The top boundary index for cropping.
        right (int): The right boundary index for cropping.
        bottom (int): The bottom boundary index for cropping.

    Returns:
        np.ndarray: The cropped portion of the input array (image).

    Raises:
        ValueError: If the input array is not 2D or 3D.
    """
    if len(arr.shape) == 2:  # Grayscale image (2D array)
        return arr[top:bottom, left:right]
    elif len(arr.shape) == 3:  # Color image (3D array)
        return arr[:, top:bottom, left:right]
    elif len(arr.shape) == 4:  # Color image (3D array)
        return arr[:, :, top:bottom, left:right]
    else:
        raise ValueError("Input array must be a 2D, 3D or 4D array")



def get_raster_data(
    fname: str | dict[str, dict[str, str]],
    is_label: bool = True,
    bands: List[int] | None = None,
    no_data_value: int | None = -9999,
    mask_cloud: bool = True,
    water_mask: bool = False,
) -> np.ndarray:
    """Load and process raster data from a file.

    Args:
        fname (str): Filename to load data from.
        is_label (bool): Whether the file is a label file.
        bands (List[int]): Index of bands to select from array.
        no_data_value (int | None): NODATA value in image raster.
        mask_cloud (bool): Perform cloud masking.
        water_mask (bool): Perform water masking.

    Returns:
        np.ndarray: Numpy array representing the processed data.
    """
    if isinstance(fname, dict):
        data, mask, crs = open_mf_tiff_dataset(fname, load_masks=False)
        data = data.fillna(no_data_value)
        data = data.band_data.values
    else:
        with rasterio.open(fname) as src:
            data = src.read()
    if (not is_label) and bands:
        data = data[bands, ...]
    # For some reasons, some few HLS tiles are not scaled in v2.0.
    # In the following lines, we find and scale them
    bands = []
    for band in data:
        band = band.astype(np.float32)
        if band.max() > 10:
            band *= 0.0001
        bands.append(band)
    data = np.stack(bands, axis=0)
    
    return data
from typing import List, Tuple
import numpy as np
import torch
from functools import partial

def process_test(
    x: np.ndarray,
    mean: List[float],
    std: List[float],
    temporal_size: int = 1,
    img_size: int = 512,
    crop_size: int = 224,
    stride: int = 224,
) -> torch.Tensor:
    """Process and augment test data without expecting masks.

    Args:
        x (np.ndarray): Input image array.
        mean (List[float]): Mean values for normalization.
        std: (List[float]): Standard deviation values for normalization.
        temporal_size (int, optional): Temporal dimension size. Defaults to 1.
        img_size (int, optional): Size of the input images. Defaults to 512.
        crop_size (int, optional): Size of the crops to be extracted from the images. Defaults to 224.
        stride (int, optional): Stride for cropping. Defaults to 224.

    Returns:
        torch.Tensor: Tensor containing the processed images.
    """
    preprocess_func = partial(
        process_and_augment,
        mean=mean,
        std=std,
        temporal_size=temporal_size,
        augment=False,
    )

    img_crops = []
    width, height = img_size, img_size

    for top in range(0, height - crop_size + 1, stride):
        for left in range(0, width - crop_size + 1, stride):
            bottom = top + crop_size
            right = left + crop_size

            img_crops.append(crop_array(x, left, top, right, bottom))

    # Process images using the preprocessing function
    samples = [preprocess_func(img, None) for img in img_crops]
    imgs = torch.stack([sample[0] for sample in samples])
    return imgs


def process_data(
    im_fname: str,
    mask_fname: str | None = None,
    no_data_value: int | None = -9999,
    reduce_to_zero: bool = False,
    replace_label: Tuple | None = None,
    bands: List[int] | None = None,
    constant_multiplier: float = 1.0,
    mask_cloud: bool = False,
) -> Tuple[np.ndarray, np.ndarray]:
    """Process image and mask data from filenames.

    Args:
        im_fname (str): Filename for the image data.
        mask_fname (str | None): Filename for the mask data.
        bands (List[int]): Indices of bands to select from array.
        no_data_value (int | None): NODATA value in image raster.
        reduce_to_zero (bool): Reduces the label index to start from Zero.
        replace_label (Tuple): Tuple of value to replace and the replacement value.
        constant_multiplier (float): Constant multiplier for image.
        mask_cloud (bool): Perform cloud masking.

    Returns:
        Tuple[np.ndarray, np.ndarray]: A tuple of numpy arrays representing the processed
        image and mask data.
    """
    arr_x = get_raster_data(
        im_fname,
        is_label=False,
        bands=bands,
        no_data_value=no_data_value,
        mask_cloud=mask_cloud,
        water_mask=False,
    )
    arr_x = arr_x * constant_multiplier
    if mask_fname:
        arr_y = get_raster_data(mask_fname)
        if replace_label:
            arr_y = np.where(arr_y == replace_label[0], replace_label[1], arr_y)
        if reduce_to_zero:
            arr_y -= 1
    else:
        arr_y = None
    return arr_x, arr_y


def load_data_from_csv(fname: str, input_root: str) -> List[Tuple[str, str | None]]:
    """Load data file paths from a CSV file.

    Args:
        fname (str): Filename of the CSV file.
        input_root (str): Root directory for input images and labels.

    Returns:
        List[Tuple[str, str]]: A list of tuples, each containing file paths for input
        image and label image.
    """
    file_paths = []
    data = pd.read_csv(fname)
    label_present = True if "Label" in data.columns else False
    for _, row in data.iterrows():
        im_path = os.path.join(input_root, row["Input"])
        mask_path = (
            None if not label_present else os.path.join(input_root, row["Label"])
        )
        if os.path.exists(im_path):
            try:
                with rasterio.open(im_path) as src:
                    _ = src.crs
                file_paths.append((im_path, mask_path))
            except Exception as e:
                logging.error(e)
                continue
    return file_paths


class InstaGeoDataset(torch.utils.data.Dataset):
    """InstaGeo PyTorch Dataset for Loading and Handling HLS Data."""

    def __init__(
        self,
        filename: str,
        input_root: str,
        preprocess_func: Callable,
        no_data_value: int | None,
        replace_label: Tuple,
        reduce_to_zero: bool,
        constant_multiplier: float,
        bands: List[int] | None = None,
        include_filenames: bool = False,
    ):
        """Dataset Class for loading and preprocessing the dataset.

        Args:
            filename (str): Filename of the CSV file containing data paths.
            input_root (str): Root directory for input images and labels.
            preprocess_func (Callable): Function to preprocess the data.
            bands (List[int]): Indices of bands to select from array.
            no_data_value (int | None): NODATA value in image raster.
            reduce_to_zero (bool): Reduces the label index to start from Zero.
            replace_label (Tuple): Tuple of value to replace and the replacement value.
            constant_multiplier (float): Constant multiplier for image.
            include_filenames (bool): Flag that determines whether to return filenames.

        """
        self.input_root = input_root
        self.preprocess_func = preprocess_func
        self.bands = bands
        self.file_paths = load_data_from_csv(filename, input_root)
        self.no_data_value = no_data_value
        self.replace_label = replace_label
        self.reduce_to_zero = reduce_to_zero
        self.constant_multiplier = constant_multiplier
        self.include_filenames = include_filenames

    def __getitem__(self, i: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """Retrieves a sample from dataset.

        Args:
            i (int): Sample index to retrieve.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: A tuple of tensors representing the
            processed images and label.
        """
        im_fname, mask_fname = self.file_paths[i]
        arr_x, arr_y = process_data(
            im_fname,
            mask_fname,
            no_data_value=self.no_data_value,
            replace_label=self.replace_label,
            reduce_to_zero=self.reduce_to_zero,
            bands=self.bands,
            constant_multiplier=self.constant_multiplier,
        )
        img_tensor, mask_tensor = self.preprocess_func(arr_x, arr_y)

        return {"image": img_tensor, "mask": mask_tensor.long()}  # 🔥 Return a dictionary

    def __len__(self) -> int:
        """Return length of dataset."""
        return len(self.file_paths)


In [11]:
## EXECUTE
dict_config = {
    "root_dir": None,
    "valid_filepath": None,
    "train_filepath": None,
    "test_filepath": None,
    "checkpoint_path": None,
    "mode": "train",  # "train" or "eval"

    "train": {
        "learning_rate": 0.0001,
        "num_epochs": 20,
        "batch_size": 16,
        "class_weights": [1, 1],
        "ignore_index": -1,
        "weight_decay": 0.1
    },

    "model": {
        "freeze_backbone": True,  # Set to False if required
        "num_classes": 2
    },

    "dataloader": {
        "bands": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
        "mean": [670.5441284179688, 1267.7974853515625, 1772.599365234375, 2415.69091796875, 2879.2431640625, 2337.822509765625],
        "std": [2146.305419921875, 2203.416259765625, 2247.03515625, 2310.74755859375, 2322.708984375, 2211.968505859375],
        "img_size": 256,
        "temporal_dim": 3,
        "replace_label": [-9999, -1],
        "reduce_to_zero": False,
        "no_data_value": -9999,
        "constant_multiplier": 1.0
    },

    "test": {
        "img_size": 256,
        "crop_size": 256,
        "stride": 256,
        "mask_cloud": False
    }
}

# Assigning values from the dictionary
BANDS = dict_config["dataloader"]["bands"]
MEAN = dict_config["dataloader"]["mean"]
STD = dict_config["dataloader"]["std"]
IM_SIZE = dict_config["dataloader"]["img_size"]
TEMPORAL_SIZE = dict_config["dataloader"]["temporal_dim"]


In [14]:
pd.read_csv("train_split.csv")

,Input,Label
0,hls_train/hls_train/chips/chip_20180401_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20180401_...
1,hls_train/hls_train/chips/chip_20180601_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20180601_...
2,hls_train/hls_train/chips/chip_20200701_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20200701_...
3,hls_train/hls_train/chips/chip_20190101_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20190101_...
4,hls_train/hls_train/chips/chip_20181101_S30_T2...,hls_train/hls_train/seg_maps/seg_map_20181101_...
...,...,...
5883,hls_train/hls_train/chips/chip_20191101_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20191101_...
5884,hls_train/hls_train/chips/chip_20200701_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20200701_...
5885,hls_train/hls_train/chips/chip_20180601_S30_T4...,hls_train/hls_train/seg_maps/seg_map_20180601_...
5886,hls_train/hls_train/chips/chip_20191201_S30_T3...,hls_train/hls_train/seg_maps/seg_map_20191201_...


In [15]:
pd.read_csv('test_ds.csv')

,Input
0,hls_test/hls_test/chips/chip_20210101_S30_T42Q...
1,hls_test/hls_test/chips/chip_20210501_S30_T42Q...
2,hls_test/hls_test/chips/chip_20210301_L30_T43R...
3,hls_test/hls_test/chips/chip_20211201_L30_T36Q...
4,hls_test/hls_test/chips/chip_20210101_S30_T37Q...
...,...
2399,hls_test/hls_test/chips/chip_20210101_S30_T38P...
2400,hls_test/hls_test/chips/chip_20211201_S30_T37Q...
2401,hls_test/hls_test/chips/chip_20210701_S30_T39Q...
2402,hls_test/hls_test/chips/chip_20210501_S30_T40Q...


In [16]:
## EXECUTE
train_filepath = "train_split.csv"
val_filepath = "validation_split.csv"
root_dir = "/kaggle/input/geo-ai-hack"
train_dataset = InstaGeoDataset(
            filename=train_filepath,
            input_root=root_dir,
            preprocess_func=partial(
                process_and_augment,
                mean=MEAN,
                std=STD,
                temporal_size=TEMPORAL_SIZE,
                im_size=IM_SIZE,
            ),
            bands=BANDS,
            replace_label=dict_config["dataloader"]['replace_label'],
            reduce_to_zero=dict_config["dataloader"]['reduce_to_zero'],
            no_data_value=dict_config["dataloader"]['no_data_value'],
            constant_multiplier=dict_config["dataloader"]['constant_multiplier'],
        )
val_dataset = InstaGeoDataset(
            filename=val_filepath,
            input_root=root_dir,
            preprocess_func=partial(
                process_and_augment,
                mean=MEAN,
                std=STD,
                temporal_size=TEMPORAL_SIZE,
                im_size=IM_SIZE,
            ),
            bands=BANDS,
            replace_label=dict_config["dataloader"]['replace_label'],
            reduce_to_zero=dict_config["dataloader"]['reduce_to_zero'],
            no_data_value=dict_config["dataloader"]['no_data_value'],
            constant_multiplier=dict_config["dataloader"]['constant_multiplier'],
        )

In [17]:
## EXECUTE
from torch.utils.data import DataLoader, Dataset
from typing import Optional
def create_dataloader(
    dataset: Dataset,
    batch_size: int,
    shuffle: bool = False,
    num_workers: int = 1,
    collate_fn: Optional[Callable] = None,
    pin_memory: bool = True,
) -> DataLoader:
    """Create a DataLoader for the given dataset.

    This function is a convenient wrapper around the PyTorch DataLoader class,
    allowing for easy setup of various DataLoader parameters.

    Args:
        dataset (Dataset): The dataset to load data from.
        batch_size (int): How many samples per batch to load.
        shuffle (bool): Set to True to have the data reshuffled at every epoch.
        num_workers (int): How many subprocesses to use for data loading.
        collate_fn (Optional[Callable]): Merges a list of samples to form a mini-batch.
        pin_memory (bool): If True, the data loader will copy tensors into CUDA pinned
            memory.

    Returns:
        DataLoader: An instance of the PyTorch DataLoader.
    """
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=pin_memory,
    )


In [18]:
## EXECUTE
batch_size = dict_config['train']['batch_size']
train_loader = create_dataloader(
            train_dataset, batch_size=batch_size, shuffle=True, num_workers=1
        )
val_loader = create_dataloader(
            val_dataset, batch_size=batch_size, shuffle=True, num_workers=1
        )

In [19]:
## EXECUTE
for batch in train_loader:

    print(f"Batch Shape: {batch['image'].shape}")
    print(f"Label Shape: {batch['mask'].shape}")
    break
print(len(train_loader))

Batch Shape: torch.Size([16, 6, 3, 256, 256])
Label Shape: torch.Size([16, 256, 256])
368


In [20]:
## EXECUTE
for batch in val_loader:
    print(f"Batch Shape: {batch['image'].shape}")
    print(f"Label Shape: {batch['mask'].shape}")
    break
print(len(val_loader))

Batch Shape: torch.Size([16, 6, 3, 256, 256])
Label Shape: torch.Size([16, 256, 256])
158


In [22]:
pip install terratorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.0/849.0 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 62.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
## EXECUTE
import torch
import torch.nn as nn
import lightning.pytorch as pl
from torch.optim import AdamW
from torch.utils.data import DataLoader
from terratorch.tasks import SemanticSegmentationTask
from einops import rearrange

class PrithviSegmentation(pl.LightningModule):
    def __init__(self, num_classes: int = 2, in_channels: int = 6, learning_rate: float = 1e-4):
        """Initialize the segmentation model using Prithvi 2.0 backbone."""
        super().__init__()
        self.learning_rate = learning_rate

        # Load Prithvi-EO-2.0-300M as a backbone
        self.model = SemanticSegmentationTask(
            model_args={
                "decoder": "UperNetDecoder",
                "backbone_pretrained": True,
                "backbone": "prithvi_eo_v2_600_tl",  
                "backbone_in_channels": in_channels,  
                "rescale": True,
                "backbone_bands": ["BLUE", "GREEN", "RED", "NIR_BROAD", "SWIR_1", "SWIR_2"],
                "backbone_num_frames": 1,
                "num_classes": num_classes,
                "head_dropout": 0.1,
                "decoder_channels": 256,
                "decoder_scale_modules": True,
                "head_channel_list": [128, 64],
                "necks": [
                    {"name": "SelectIndices", "indices": [7, 15, 23, 31]},
                    {"name": "ReshapeTokensToImage"},
                ],
            },
            plot_on_val=False,
            loss="focal",
            lr=self.learning_rate,
            optimizer="AdamW",
            optimizer_hparams={"weight_decay": 0.1},
            scheduler="StepLR",
            scheduler_hparams={"step_size": 10, "gamma": 0.9},
            ignore_index=-1,
            freeze_backbone=False,
            freeze_decoder=False,
            model_factory="EncoderDecoderFactory",
        )

    


In [26]:
## EXECUTE
# Import PyTorch Lightning Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from terratorch.tasks import SemanticSegmentationTask
# Logger
logger = TensorBoardLogger(save_dir="desert_locust", name="segmentation")

# Checkpoint Callback
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    filename="epoch-{epoch:02d}",
    save_top_k=-1,
    every_n_epochs=1,
    save_on_train_epoch_end=True
)

# Setup Trainer
model = SemanticSegmentationTask(
            model_args={
        "decoder": "UperNetDecoder",
        "backbone_pretrained": True,
        "backbone": "prithvi_eo_v2_300_tl", # Model can be either prithvi_eo_v2_300, prithvi_eo_v2_300_tl, prithvi_eo_v2_600, prithvi_eo_v2_600_tl
        "backbone_in_channels": 6,
        "backbone_features_only": True,
        # "backbone_coords_encoding": ["time", "location"], # this must be used to use time and location metadata
        "rescale": True,
        "backbone_bands": ["BLUE", "GREEN", "RED", "NIR_NARROW", "SWIR_1", "SWIR_2"],
        "backbone_num_frames": 3,
        "num_classes": 2,
        "head_dropout": 0.1,
        "decoder_channels": 256,
        "decoder_scale_modules": True,
        "necks": [
            {
                "name": "SelectIndices",
                #"indices": [2, 5, 8, 11] # indices for prithvi_vit_100
                "indices": [5, 11, 17, 23] # indices for prithvi_eo_v2_300
                #"indices": [7, 15, 23, 31] # indices for prithvi_eo_v2_600
            },
            {
                "name": "ReshapeTokensToImage",
                "effective_time_dim": 3
            }
        ]
            },
            plot_on_val=False,
            loss="ce",
            lr=1.0e-4,
            optimizer="AdamW",
            optimizer_hparams={"weight_decay": 0.1},
            ignore_index=-1,
            freeze_backbone=True,
            freeze_decoder=False,
            model_factory="EncoderDecoderFactory",
        )
trainer = pl.Trainer(
    accelerator="auto",
    strategy="auto",
    devices="auto",
    num_nodes=1,
    logger = logger,
    max_epochs=50,
    check_val_every_n_epoch=1,
    log_every_n_steps=1,
    enable_checkpointing=True,
    callbacks=[checkpoint_callback],
    default_root_dir="root_dir",
)

# Train Model
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader) #, ckpt_path="/kaggle/working/epoch-epoch=00.ckpt" )

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (('ibm-nasa-geospatial/Prithvi-EO-2.0-300M-TL', 'Prithvi_EO_V2_300M_TL.pt'))


In [27]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
import rasterio
import os
import torch

class TestDataset(Dataset):
    """Dataset class for test images (without labels)."""
    def __init__(self, test_image_paths):
        self.test_image_paths = test_image_paths

    def __len__(self):
        return len(self.test_image_paths)

    def __getitem__(self, idx):
        image_path = self.test_image_paths[idx]
        with rasterio.open(image_path) as src:
            image = src.read()  # Read image as numpy array (C, H, W)
        image = torch.tensor(image, dtype=torch.float32) 
        image = image.view(6,3,256,256)
        return image, image_path  # Return image and filename

# Load test image paths
test_images_dir = "extracted_files/hls_test/chips"
test_image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir)]

# Create test dataloader
test_dataset = TestDataset(test_image_paths)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [28]:
for batch in test_loader:
    print(batch[0].shape)
    break

torch.Size([1, 6, 3, 256, 256])


In [41]:
import torch
import os
import rasterio
import numpy as np
import pandas as pd
from pyproj import Transformer
from rasterio.transform import AffineTransformer
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from terratorch.tasks import SemanticSegmentationTask

# Define the checkpoint path
best_ckpt_path = "epoch-epoch=01.ckpt"  # Replace XX with the best epoch

# Load the model from checkpoint
model = SemanticSegmentationTask.load_from_checkpoint(
    best_ckpt_path,
    model_args=model.hparams.model_args,
    model_factory=model.hparams.model_factory
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (('ibm-nasa-geospatial/Prithvi-EO-2.0-300M-TL', 'Prithvi_EO_V2_300M_TL.pt'))


SemanticSegmentationTask(
  (model): PixelWiseModel(
    (encoder): TimmBackboneWrapper(
      (_timm_module): PrithviViT(
        (patch_embed): PatchEmbed(
          (proj): Conv3d(6, 1024, kernel_size=(1, 16, 16), stride=(1, 16, 16))
          (norm): Identity()
        )
        (temporal_embed_enc): TemporalEncoder()
        (location_embed_enc): LocationEncoder()
        (blocks): ModuleList(
          (0-23): 24 x Block(
            (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attn): Attention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (q_norm): Identity()
              (k_norm): Identity()
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): Identity()
            (drop_path1): Identity()
            (norm2): LayerNorm((1024,), eps

In [31]:
for image, filename in test_loader:
    print(image.shape)
    print(filename)
    break

torch.Size([1, 6, 3, 256, 256])
('extracted_files/hls_test/chips/chip_20210101_S30_T36QXD_2020338T081319_12_0.tif',)


In [42]:
import numpy as np
from tqdm import tqdm
predictions_dir = "predictions"
os.makedirs(predictions_dir, exist_ok=True)

with torch.no_grad():
    for image, filename in tqdm(test_loader):
        image = image.to(device)
        output = model(image)
        pred_mask = torch.argmax(output.output, dim=1).cpu().numpy()[0]  # Get predicted mask

        # Save prediction as a raster
        save_path = os.path.join(predictions_dir, os.path.basename(filename[0]).replace(".tif", "_pred.tif"))
        with rasterio.open(filename[0]) as src:
            meta = src.meta.copy()
            meta.update(dtype=rasterio.int16, nodata=-9999)
            with rasterio.open(save_path, "w", **meta) as dst:
                dst.write(pred_mask.astype(np.uint8), 1)


100%|██████████| 2404/2404 [1:03:07<00:00,  1.58s/it]


In [ ]:
print(len(test_loader))
print(len(os.listdir("/kaggle/working/predictions_s2")))

In [ ]:
import pandas as pd
from rasterio.transform import AffineTransformer
from pyproj import Transformer
from rasterio.crs import CRS

# Read test CSV
submission_df = pd.read_csv("/kaggle/input/geo-ai-hack/test.csv")

def get_prediction_value(row):
    """Find the predicted value for each test location."""
    matching_files = [f for f in os.listdir(predictions_dir) if (str(row['date']) in f) and (row['mgrs_tile_id'] in f)]
    if not matching_files:
        return np.nan

    for file in matching_files:
        with rasterio.open(os.path.join(predictions_dir, file)) as src:
            width, height = src.width, src.height
            affine_transform = AffineTransformer(src.transform)
            transformer = Transformer.from_crs(CRS.from_epsg(4326), src.crs, always_xy=True)
            x_chip, y_chip = transformer.transform(row['x'], row['y'])
            x_offset, y_offset = affine_transform.rowcol(x_chip, y_chip)

            if 0 <= x_offset < width and 0 <= y_offset < height:
                return src.read(1)[y_offset, x_offset]

    return np.nan

# Apply function to get predictions
submission_df["prediction"] = submission_df.apply(get_prediction_value, axis=1)

# Save final submission
submission_df[["id", "prediction"]].to_csv("s2_submission.csv", index=False)

print("✅ Submission file saved as `hls_submission.csv`")


In [ ]:
submission_df['prediction'].unique()

In [ ]:
submission_df[["id", "prediction"]].to_csv("s2_submission.csv", index=False)